# Week 4 — Azure SQL Integration & Data Ingestion

This notebook connects the predictive maintenance dataset to **Azure SQL Database**
(`predictive_maintenance_db` on `goitom-pm-sqlserver.database.windows.net`).

**Goals:**
1. Load the engineered dataset from `../data/dataset_ready_for_model.csv`
2. Connect securely to Azure SQL using `pyodbc`
3. Automatically create a table `sensor_features` with matching schema
4. Bulk-insert all rows into Azure SQL
5. Verify the data by running a `SELECT` from Azure SQL

In [1]:
import pyodbc
import pandas as pd
from pathlib import Path

# === 1. Load engineered dataset ===
data_path = Path("..") / "data" / "dataset_ready_for_model.csv"
df = pd.read_csv(data_path)

print(f"Loaded dataset with shape: {df.shape}")
df.head()

Loaded dataset with shape: (42740, 25)


,machine_id,reading_time,temperature,vibration,pressure,current,rpm,status_code,temp_mean_6h,temp_std_6h,...,vib_std_12h,temp_mean_24h,temp_std_24h,vib_mean_24h,vib_std_24h,temp_delta_1h,vib_delta_1h,temp_delta_6h,vib_delta_6h,failure_within_72h
0,1,2023-01-01 23:00:00,57.150504,0.937167,30.242980,9.870475,1563.794434,0,59.187442,2.202235,...,0.106564,59.704723,1.947517,1.060162,0.089745,-2.984553,-0.133819,-3.477991,0.036166,0
1,1,2023-01-02 00:00:00,58.911235,1.126460,29.902485,10.259461,1503.926682,0,59.308655,2.155474,...,0.100237,59.617965,1.933944,1.065017,0.090061,1.760731,0.189293,0.727283,0.056084,0
2,1,2023-01-02 01:00:00,60.221845,0.997305,30.119151,9.905707,1437.663984,0,59.816397,1.895608,...,0.101074,59.638731,1.937798,1.060546,0.090671,1.310611,-0.129155,3.046453,0.035122,0
3,1,2023-01-02 02:00:00,57.698013,0.987546,29.613272,9.712392,1431.948007,0,58.944183,1.279613,...,0.102235,59.488841,1.943205,1.053577,0.089533,-2.523832,-0.009759,-5.233285,-0.284031,0
4,1,2023-01-02 03:00:00,60.751396,0.989948,30.191727,10.337799,1480.568773,0,59.144675,1.472871,...,0.099032,59.393230,1.812636,1.053666,0.089465,3.053383,0.002401,1.202949,-0.035349,0


## Azure SQL connection

We use the ADO.NET (SQL authentication) connection string from the Azure portal,
adapted for `pyodbc`.

Replace **YOUR_PASSWORD_HERE** with the actual password you set for `sqladmin`.


In [2]:
import os
import pyodbc

server   = "goitom-pm-sqlserver.database.windows.net"
database = "predictive_maintenance_db"
username = "sqladmin"
password = os.getenv("AZURE_SQL_PASSWORD")

if password is None:
    raise ValueError("❌ Environment variable AZURE_SQL_PASSWORD is not set")

conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    f"SERVER={server},1433;"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
    "Encrypt=yes;"
    "TrustServerCertificate=no;"
    "Connection Timeout=30;"
)

conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

print("Connected to Azure SQL successfully (password hidden).")


Connected to Azure SQL successfully (password hidden).


## Auto-create `sensor_features` table

To avoid mismatches between the CSV columns and SQL schema, we infer the SQL
column types from the pandas `DataFrame` dtypes and generate a `CREATE TABLE`
statement automatically.

Table name: **sensor_features**

- `id` INT IDENTITY primary key
- One column per feature in the dataset

In [3]:
# === 3. Generate CREATE TABLE statement dynamically ===

table_name = "sensor_features"

def map_dtype_to_sql(dtype) -> str:
    """Map pandas dtypes to Azure SQL types."""
    if pd.api.types.is_integer_dtype(dtype):
        return "INT"
    if pd.api.types.is_float_dtype(dtype):
        return "FLOAT"
    if pd.api.types.is_bool_dtype(dtype):
        return "BIT"
    if pd.api.types.is_datetime64_any_dtype(dtype):
        return "DATETIME2"
    # default to NVARCHAR for anything else (object, string, etc.)
    return "NVARCHAR(255)"

# Build column definitions from df dtypes
columns_sql = []
for col, dtype in df.dtypes.items():
    sql_type = map_dtype_to_sql(dtype)
    safe_col = col.replace(" ", "_")  # avoid spaces in column names
    columns_sql.append(f"[{safe_col}] {sql_type}")

columns_sql_str = ",\n    ".join(columns_sql)

create_table_sql = f"""
IF OBJECT_ID('{table_name}', 'U') IS NOT NULL
    DROP TABLE {table_name};

CREATE TABLE {table_name} (
    id INT IDENTITY(1,1) PRIMARY KEY,
    {columns_sql_str}
);
"""

print(create_table_sql)



IF OBJECT_ID('sensor_features', 'U') IS NOT NULL
    DROP TABLE sensor_features;

CREATE TABLE sensor_features (
    id INT IDENTITY(1,1) PRIMARY KEY,
    [machine_id] INT,
    [reading_time] NVARCHAR(255),
    [temperature] FLOAT,
    [vibration] FLOAT,
    [pressure] FLOAT,
    [current] FLOAT,
    [rpm] FLOAT,
    [status_code] INT,
    [temp_mean_6h] FLOAT,
    [temp_std_6h] FLOAT,
    [vib_mean_6h] FLOAT,
    [vib_std_6h] FLOAT,
    [temp_mean_12h] FLOAT,
    [temp_std_12h] FLOAT,
    [vib_mean_12h] FLOAT,
    [vib_std_12h] FLOAT,
    [temp_mean_24h] FLOAT,
    [temp_std_24h] FLOAT,
    [vib_mean_24h] FLOAT,
    [vib_std_24h] FLOAT,
    [temp_delta_1h] FLOAT,
    [vib_delta_1h] FLOAT,
    [temp_delta_6h] FLOAT,
    [vib_delta_6h] FLOAT,
    [failure_within_72h] INT
);



In [4]:
# Execute the CREATE TABLE statement
cursor.execute(create_table_sql)
conn.commit()

print(f"Table `{table_name}` created (or recreated) successfully.")

Table `sensor_features` created (or recreated) successfully.


## Insert data into Azure SQL

We now insert all rows from the pandas DataFrame into the `sensor_features` table.
We use `executemany` for faster bulk insert.

In [ ]:
# === 4. Bulk insert rows ===

# Use the same column order as in df, but with spaces replaced by underscores
safe_columns = [c.replace(" ", "_") for c in df.columns]

placeholders = ", ".join("?" for _ in safe_columns)
columns_sql_list = ", ".join(f"[{c}]" for c in safe_columns)

insert_sql = f"""
INSERT INTO {table_name} ({columns_sql_list})
VALUES ({placeholders});
"""

print("Insert SQL preview:")
print(insert_sql)

# Convert DataFrame rows to list of tuples
records = []
for _, row in df.iterrows():
    records.append(tuple(row.values))

print(f"Prepared {len(records)} records for insertion.")

# Execute in batches to avoid memory issues
batch_size = 1000
for start in range(0, len(records), batch_size):
    end = start + batch_size
    batch = records[start:end]
    cursor.executemany(insert_sql, batch)
    conn.commit()
    print(f"Inserted rows {start}–{end-1}")

print("All rows inserted into Azure SQL.")

Insert SQL preview:

INSERT INTO sensor_features ([machine_id], [reading_time], [temperature], [vibration], [pressure], [current], [rpm], [status_code], [temp_mean_6h], [temp_std_6h], [vib_mean_6h], [vib_std_6h], [temp_mean_12h], [temp_std_12h], [vib_mean_12h], [vib_std_12h], [temp_mean_24h], [temp_std_24h], [vib_mean_24h], [vib_std_24h], [temp_delta_1h], [vib_delta_1h], [temp_delta_6h], [vib_delta_6h], [failure_within_72h])
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);

Prepared 42740 records for insertion.
Inserted rows 0–999
Inserted rows 1000–1999
Inserted rows 2000–2999
Inserted rows 3000–3999
Inserted rows 4000–4999
Inserted rows 5000–5999
Inserted rows 6000–6999


## Verify data in Azure SQL

We run a simple `SELECT COUNT(*)` and top 5 rows to confirm that the data
was saved correctly in the `sensor_features` table.

In [ ]:
# === 5. Verify row count ===
cursor.execute(f"SELECT COUNT(*) FROM {table_name};")
row_count = cursor.fetchone()[0]
print(f"Row count in `{table_name}`: {row_count}")

# Show a few rows
cursor.execute(f"SELECT TOP 5 * FROM {table_name};")
rows = cursor.fetchall()
for r in rows:
    print(r)

In [ ]:
# === 6. Cleanup ===
cursor.close()
conn.close()
print("Connection closed.")